# Controls and scripting

In this notebook we will see a powerful way of modifying dynamically `BendingCallback` parameters with `BendingParameter`, that automatically handles export while scripting a model (typically for a use in [nn~](https://github.com/acids-ircam/nn_tilde)).

## Callbacks and Parameters

Every bending operations in `torchbend` are objects inheriting from `BendingCallback`, being itself a `nn.Module`. This way, the callback can record buffers and various attributes required for its insertion into a model scripted through `torch.jit`. For this reason, `BendingParameter` can be used to dynamically modify the parameters of one or several callbacks and perform basic arithmetics : 

In [1]:
import torch, torch.nn as nn
import sys; sys.path.append("..")
import torchbend as tb

class Greg(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv_module_1 = nn.Conv1d(1, 4, 3)
        self.conv_module_2 = nn.Conv1d(4, 1, 3)
    def forward(self, x):
        out_1 = self.conv_module_1(x)
        out_2 = self.conv_module_2(out_1)
        return out_2

module = Greg()
bended = tb.BendedModule(module)
x = torch.zeros(1, 1, 16)
graph, out = bended.trace(x=x, _return_out=True)
print('-- Original output : \n', out[0])

# initialize bending paramteer with a name and a value
c1 = tb.BendingParameter("macro1", value=1.)
bended.bend(tb.Scale(1. * c1), "conv_module_1$")
bended.bend(tb.Scale(2. * c1), "conv_module_2$")

print(bended.controllables)

for i in [-0.5, 0., 1., 2.]:
    bended.update(c1.name, i)
    out = bended(x)
    print(f'-- output with c1 = {i} : \n', out[0]) 




-- Original output : 
 tensor([[[0.1299, 0.1299, 0.1299, 0.1299, 0.1299, 0.1299, 0.1299, 0.1299,
          0.1299, 0.1299, 0.1299, 0.1299]]], grad_fn=<ConvolutionBackward0>)
{'macro1': BendingParameter(name=macro1, value=Parameter containing:
tensor(1.))}
-- output with c1 = -0.5 : 
 tensor([[-0.0649, -0.0649, -0.0649, -0.0649, -0.0649, -0.0649, -0.0649, -0.0649,
         -0.0649, -0.0649, -0.0649, -0.0649]], grad_fn=<SelectBackward0>)
-- output with c1 = 0.0 : 
 tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],
       grad_fn=<SelectBackward0>)
-- output with c1 = 1.0 : 
 tensor([[0.1299, 0.1299, 0.1299, 0.1299, 0.1299, 0.1299, 0.1299, 0.1299, 0.1299,
         0.1299, 0.1299, 0.1299]], grad_fn=<SelectBackward0>)
-- output with c1 = 2.0 : 
 tensor([[0.2597, 0.2597, 0.2597, 0.2597, 0.2597, 0.2597, 0.2597, 0.2597, 0.2597,
         0.2597, 0.2597, 0.2597]], grad_fn=<SelectBackward0>)


Thus, `BendingParameter` allows to define somehow "macros" on our bending operations. Besides being useful, we will see below how `BendingParameter` also significantly eases bending operations during scripting. 

## Scripting

`BendedModule` can also leverage the `torch.fx.GraphModule` structure to perform automatic scripting of bended modules, provided that the target functions can be scripted. Automatic scripting is processed as follows :

![scripting process](img/scripting.png "Scripting process")

Automatic `BendingParameter` setter and getters are also added to the final `ScriptedBendedModule`, allowing to modify the values of the macros after scripting.

In [5]:
class Greg(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv_module_1 = nn.Conv1d(1, 4, 3)
        self.conv_module_2 = nn.Conv1d(4, 1, 3)
    def forward(self, x):
        out_1 = self.conv_module_1(x)
        out_2 = self.conv_module_2(out_1)
        return out_2
    @torch.jit.export
    def half_forward(self, x):
        out_1 = self.conv_module_1(x)
        return out_1

module = Greg()
bended = tb.BendedModule(module)

x = torch.zeros(1, 1, 16)
bended.trace(x=x, _return_out=True)
# a range argument can be added to constrain value to a certain range
c1 = tb.BendingParameter("macro1", value=1., range=[-1., 1.])
bended.bend(tb.Scale(1. * c1), "conv_module_1$")
bended.bend(tb.Scale(2. * c1), "conv_module_2$")

scripted = bended.script()
out = scripted(x)
print(out)
scripted.set_macro1(0.)
out = scripted(x)
print(out)
try:
    scripted.set_macro1(-3.)
except Exception as e:
    # an error is issued by the torchscript interpreted if the value is outside the range
    print("setting failed : ", e)

tensor([[[0.0423, 0.0423, 0.0423, 0.0423, 0.0423, 0.0423, 0.0423, 0.0423,
          0.0423, 0.0423, 0.0423, 0.0423]]], grad_fn=<MulBackward0>)
tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]],
       grad_fn=<MulBackward0>)
setting failed :  The following operation failed in the TorchScript interpreter.
Traceback of TorchScript (most recent call last):
  File "/Users/domkirke/Dropbox/code/torchbend/docs/../torchbend/tracing/script.py", line 166, in set_macro1
        for v in self._controllables:
            if v.name == name:
                v.set_value(value)
                ~~~~~~~~~~~ <--- HERE
        self._update_weights(name)
        return 0
  File "/Users/domkirke/Dropbox/code/torchbend/docs/../torchbend/bending/parameter.py", line 147, in set_value
            if self.min_clamp is not None:
                if value < self.min_clamp:
                    raise BendingParameterException(f'tried to set value < min_clamp = {self.min_clamp}, but got {value}')
           

`Be